In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

from scrappers import WhoScored
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Output directory
os.makedirs('../data', exist_ok=True)

In [ ]:
# SEASON 25/26
print("="*60)
print("BARCELONA 25/26 - PRIMEROS 14 PARTIDOS DE LA LIGA")
print("="*60)

ws_2526 = WhoScored(leagues=["ESP-La Liga"], seasons=["25-26"])
schedule_2526 = ws_2526.read_schedule()

schedule_2526_reset = schedule_2526.reset_index()
barcelona_2526 = schedule_2526_reset[
    (schedule_2526_reset['home_team'].str.contains('Barcelona', case=False, na=False)) |
    (schedule_2526_reset['away_team'].str.contains('Barcelona', case=False, na=False))
].copy()

barcelona_2526 = barcelona_2526[['date', 'home_team', 'away_team', 'game_id']].copy()
barcelona_2526 = barcelona_2526.rename(columns={'game_id': 'match_id'})
barcelona_2526 = barcelona_2526.sort_values('date').reset_index(drop=True)

# First 14 matches only
barcelona_2526_first14 = barcelona_2526.head(14)

print(f"Total partidos Barcelona en La Liga 25/26: {len(barcelona_2526)}")
print(f"Primeros 14 partidos:")
barcelona_2526_first14

In [ ]:
# Scrape events for 25/26
csv_path_2526 = '../data/barcelona_events_2526_first14.csv'

if os.path.exists(csv_path_2526):
    print(f"Cargando desde CSV: {csv_path_2526}")
    df_2526_events = pd.read_csv(csv_path_2526)
    print(f"Cargados: {len(df_2526_events)} eventos")
else:
    print("Scrapeando eventos 25/26...")
    all_events_2526 = []
    
    for idx, match in barcelona_2526_first14.iterrows():
        print(f"\n[{idx+1}/14] {match['home_team']} vs {match['away_team']}")
        
        try:
            events = ws_2526.read_events(match_id=int(match['match_id']))
            
            # Extract ALL events (both teams)
            if len(events) > 0:
                events['match_date'] = match['date']
                events['opponent'] = match['away_team'] if 'Barcelona' in match['home_team'] else match['home_team']
                events['venue'] = 'Home' if 'Barcelona' in match['home_team'] else 'Away'
                events['match_id'] = match['match_id']
                events['match_number'] = idx + 1
                all_events_2526.append(events)
                print(f"  {len(events)} eventos totales")
            else:
                print(f"  Sin eventos")
                
        except Exception as e:
            print(f"  Error: {e}")
            continue
    
    if len(all_events_2526) > 0:
        df_2526_events = pd.concat(all_events_2526, ignore_index=True)
        df_2526_events.to_csv(csv_path_2526, index=False)
        print(f"\nGuardado: {csv_path_2526}")
        print(f"Total eventos: {len(df_2526_events)}")
    else:
        df_2526_events = pd.DataFrame()

print(f"\nResumen 25/26:")
print(f"Total eventos: {len(df_2526_events)}")
if len(df_2526_events) > 0:
    print(f"\nTipos de eventos:")
    print(df_2526_events['type'].value_counts().head(10))

In [ ]:
# Final summary
print("\n" + "="*60)
print("EXTRACCIÓN COMPLETADA")
print("="*60)
print(f"\nArchivo generado:")
print(f"  {csv_path_2526}")
print(f"  Eventos: {len(df_2526_events) if 'df_2526_events' in dir() else 'N/A'}")